In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [13]:
sub = 'sub-12'
ses = 'ses-01'
battery = 'camcan'
task = 'emotional-memory'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery, ses)
os.chdir(datadir)
csv_files = sorted(glob.glob(task + '*.txt'))
tsv_files = []
i = 0
# for i in range(len(csv_files)):
tsv_files.append(task + '_' + sub + '_run-0' + str(i+1) + '.tsv')

In [3]:
csv_files

['emotional-memory_sub-12_run-01_1624259125.txt',
 'emotional-memory_sub-12_run-02_1624259822.txt']

In [14]:
# In the log file run start is the file line.
# To extract it read in the first line by itself and then 
# read the rest of the file as a DataFrame.
line1 = pd.read_csv(os.path.join(datadir, csv_files[i]), sep=':', header=None, nrows=1)
run_start = line1[1][0]

In [15]:
run_start

1289.5722

In [16]:
file_df = pd.read_csv(os.path.join(datadir, csv_files[i]), header=1, sep=',', skipinitialspace=True)
# file_df = pd.read_csv(os.path.join(datadir, csv_files[0]), sep=',')

In [17]:
# Find the trial-related rows
# trials = file_df['img_valence'] != 'Scanner'
valence = file_df['img_valence']
negative_trials = file_df['img_valence'].isin(['3'])
neutral_trials = file_df['img_valence'].isin(['2'])
positive_trials = file_df['img_valence'].isin(['1'])

In [18]:
# Trial/Image onsets are in 'img_onset'
img_onsets = file_df['img_onset'].astype(float)

# Object onsets are in 'obj_onset'
obj_onsets = file_df['obj_onset'].astype(float)

# Concatenate the two to get the onsets array
trial_onsets = np.vstack((img_onsets, obj_onsets)).flatten('F')

## Create durations array
# In the task, the image is presented for 8 s. 2 s after image onset, 
# the object is presented, and stays on screen for the remaining 6 s.
img_durations = np.repeat(8,len(file_df))
obj_durations = np.repeat(6,len(file_df))
# Interlace the two durations to create the durations array
durations = (np.vstack((img_durations,obj_durations))).flatten('F')

In [19]:
## Finally, create the trial_types array. This will be an interlacing
## of the 'img_valence' vector for images and 'object' for objects
img_types = np.array(np.ones(len(file_df)), dtype=np.object_)
img_types[negative_trials] = 'negative_image'
img_types[neutral_trials] = 'neutral_image'
img_types[positive_trials] = 'positive_image'

In [20]:
obj_types = np.repeat('object', len(file_df))

In [21]:
trial_types = np.vstack((img_types,obj_types)).flatten('F')

In [22]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(trial_onsets-run_start), 'duration':durations, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_files[0]), sep='\t', float_format='%0.3f', index=False)